In [1]:
import pandas as pd
books = pd.read_csv("books_with_categories.csv")

In [2]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None, device="mps")
classifier("I love this")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use mps


[[{'label': 'joy', 'score': 0.9845667481422424},
  {'label': 'surprise', 'score': 0.004927210509777069},
  {'label': 'sadness', 'score': 0.004531432408839464},
  {'label': 'neutral', 'score': 0.003475322388112545},
  {'label': 'anger', 'score': 0.0013875329168513417},
  {'label': 'disgust', 'score': 0.0007134057814255357},
  {'label': 'fear', 'score': 0.0003984913637395948}]]

In [3]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [4]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548416018486023},
  {'label': 'neutral', 'score': 0.16985200345516205},
  {'label': 'sadness', 'score': 0.11640852689743042},
  {'label': 'surprise', 'score': 0.020700637251138687},
  {'label': 'disgust', 'score': 0.0191007312387228},
  {'label': 'joy', 'score': 0.015161268413066864},
  {'label': 'anger', 'score': 0.0039351521991193295}]]

In [5]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296029925346375},
  {'label': 'neutral', 'score': 0.1403854787349701},
  {'label': 'fear', 'score': 0.06816214323043823},
  {'label': 'joy', 'score': 0.04794241860508919},
  {'label': 'anger', 'score': 0.009156347252428532},
  {'label': 'disgust', 'score': 0.002628472400829196},
  {'label': 'sadness', 'score': 0.0021221612114459276}],
 [{'label': 'neutral', 'score': 0.44937244057655334},
  {'label': 'disgust', 'score': 0.27359044551849365},
  {'label': 'joy', 'score': 0.10908252000808716},
  {'label': 'sadness', 'score': 0.09362712502479553},
  {'label': 'anger', 'score': 0.04047815129160881},
  {'label': 'surprise', 'score': 0.026970233768224716},
  {'label': 'fear', 'score': 0.006879057269543409}],
 [{'label': 'neutral', 'score': 0.6462159156799316},
  {'label': 'sadness', 'score': 0.2427334487438202},
  {'label': 'disgust', 'score': 0.04342266544699669},
  {'label': 'surprise', 'score': 0.028300514444708824},
  {'label': 'joy', 'score': 0.01421144

In [6]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [7]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [8]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296029925346375},
 {'label': 'neutral', 'score': 0.1403854787349701},
 {'label': 'fear', 'score': 0.06816214323043823},
 {'label': 'joy', 'score': 0.04794241860508919},
 {'label': 'anger', 'score': 0.009156347252428532},
 {'label': 'disgust', 'score': 0.002628472400829196},
 {'label': 'sadness', 'score': 0.0021221612114459276}]

In [9]:
sorted(predictions[0],key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156347252428532},
 {'label': 'disgust', 'score': 0.002628472400829196},
 {'label': 'fear', 'score': 0.06816214323043823},
 {'label': 'joy', 'score': 0.04794241860508919},
 {'label': 'neutral', 'score': 0.1403854787349701},
 {'label': 'sadness', 'score': 0.0021221612114459276},
 {'label': 'surprise', 'score': 0.7296029925346375}]

In [12]:
import numpy as np
emotion_labels = ["anger", "disgust","fear","joy","sadness","surprise","neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}
def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index,label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label,scores in per_emotion_scores.items()}

In [13]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [14]:
emotion_scores

{'anger': [np.float64(0.06413350999355316),
  np.float64(0.6126182675361633),
  np.float64(0.06413350999355316),
  np.float64(0.351483017206192),
  np.float64(0.08141236007213593),
  np.float64(0.23222514986991882),
  np.float64(0.538184404373169),
  np.float64(0.06413350999355316),
  np.float64(0.30067071318626404),
  np.float64(0.06413350999355316)],
 'disgust': [np.float64(0.27359044551849365),
  np.float64(0.3482853174209595),
  np.float64(0.10400652140378952),
  np.float64(0.15072235465049744),
  np.float64(0.18449527025222778),
  np.float64(0.7271745800971985),
  np.float64(0.15585501492023468),
  np.float64(0.10400652140378952),
  np.float64(0.27948126196861267),
  np.float64(0.17792677879333496)],
 'fear': [np.float64(0.9281688332557678),
  np.float64(0.942527711391449),
  np.float64(0.9723208546638489),
  np.float64(0.3607076108455658),
  np.float64(0.09504329413175583),
  np.float64(0.05136270076036453),
  np.float64(0.747428297996521),
  np.float64(0.4044971466064453),
  np.

In [17]:
from tqdm import tqdm
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}
for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [03:47<00:00, 22.80it/s]


In [18]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [19]:
emotions_df.head()

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273590,0.928169,0.932798,0.646216,0.967157,0.729603,9780002005883
1,0.612618,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351483,0.150722,0.360708,0.251881,0.732686,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765,9780006280934


In [20]:
books = pd.merge(books,emotions_df,on="isbn13")

In [21]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title__and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273590,0.928169,0.932798,0.646216,0.967157,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150722,0.360708,0.251881,0.732686,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148209,0.030642,0.919165,0.255169,0.853722,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339216,0.947779,0.375756,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459268,0.759455,0.951104,0.368110,0.078765


In [22]:
books.to_csv("books_with_emotions.csv", index=False)